<a href="https://colab.research.google.com/github/UtpalikaAcharya/Integration-of-text-to-speech-TTS-generated-multilingual-data-for-LID/blob/main/tacotron2chineseTranscript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, explicitly set the locale
!apt-get update && apt-get install -y locales
!locale-gen en_US.UTF-8
!export LANG=en_US.UTF-8
!export LANGUAGE=en_US:en
!export LC_ALL=en_US.UTF-8

# Now install the required packages
!pip install --upgrade pip
!pip install TTS
!pip install torch torchaudio

# Import required libraries
import os
import sys
import torch
import warnings
from TTS.api import TTS
from IPython.display import Audio
from google.colab import drive
import pandas as pd
from tqdm.notebook import tqdm

# Set environment variables for UTF-8
os.environ['PYTHONIOENCODING'] = 'utf-8'
os.environ['LANG'] = 'en_US.UTF-8'
os.environ['LC_ALL'] = 'en_US.UTF-8'

# Suppress warnings
warnings.filterwarnings('ignore')

# Mount Google Drive
drive.mount('/content/drive')

# Your specified paths
INPUT_FOLDER = '/content/drive/MyDrive/Audio_and_Transcripts/transcript2/transcript/Transcript chinese'
OUTPUT_FOLDER = '/content/drive/MyDrive/Audio_and_Transcripts/Machine Generated Audios'
SPEAKER_WAV = "/content/drive/MyDrive/Audio_and_Transcripts/downloaded audio/downloaded audio/manderin chinese/51.wav"

# Create output directory
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Set up device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

try:
    # Initialize TTS with Chinese model
    tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)
    print("Model loaded successfully!")
except Exception as e:
    print(f"Error loading model: {e}")
    sys.exit(1)

def generate_chinese_speech(text, output_file, speaker_file):
    """Generate speech from Chinese text"""
    try:
        tts.tts_to_file(
            text=text,
            file_path=output_file,
            speaker_wav=speaker_file,
            language="zh"
        )
        return True
    except Exception as e:
        print(f"Error generating speech: {e}")
        return False

def process_files():
    """Process all text files in the input folder"""
    # Verify speaker file exists
    if not os.path.exists(SPEAKER_WAV):
        print(f"Speaker reference file not found at: {SPEAKER_WAV}")
        return

    # Get list of text files
    try:
        files = [f for f in os.listdir(INPUT_FOLDER) if f.endswith('.txt')]
    except Exception as e:
        print(f"Error accessing input folder: {e}")
        return

    if not files:
        print("No .txt files found in input folder")
        return

    results = []

    for file in tqdm(files, desc="Processing files"):
        try:
            # Read text file with explicit UTF-8 encoding
            with open(os.path.join(INPUT_FOLDER, file), 'r', encoding='utf-8') as f:
                text = f.read().strip()

            output_file = os.path.join(OUTPUT_FOLDER, f"{os.path.splitext(file)[0]}.wav")

            success = generate_chinese_speech(text, output_file, SPEAKER_WAV)

            results.append({
                'file': file,
                'status': 'success' if success else 'failed',
                'output': output_file if success else None
            })

        except Exception as e:
            print(f"Error processing {file}: {e}")
            results.append({
                'file': file,
                'status': 'failed',
                'error': str(e)
            })

    # Save processing report
    df = pd.DataFrame(results)
    report_path = os.path.join(OUTPUT_FOLDER, 'processing_report.csv')
    df.to_csv(report_path, index=False)

    return df

# Test with a single Chinese text first
def run_test():
    print("Running test...")
    test_text = "你好，这是一个测试。"
    test_output = os.path.join(OUTPUT_FOLDER, "test_output.wav")

    if generate_chinese_speech(test_text, test_output, SPEAKER_WAV):
        print("Test successful!")
        return Audio(test_output)
    else:
        print("Test failed!")
        return None

# Execute the code
print("Starting test...")
test_result = run_test()
if test_result:
    display(test_result)

    print("\nStarting main processing...")
    results = process_files()

    if results is not None:
        print("\nProcessing Summary:")
        print(f"Total files processed: {len(results)}")
        print(f"Successful conversions: {len(results[results['status'] == 'success'])}")
        print(f"Failed conversions: {len(results[results['status'] == 'failed'])}")

        # Show failed files if any
        failed = results[results['status'] == 'failed']
        if len(failed) > 0:
            print("\nFailed files:")
            print(failed[['file', 'error']])

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

Error loading model: Failed to import transformers.models.gpt2.modeling_gpt2 because of the following error (look up to see its traceback):
initialization of _pywrap_checkpoint_reader raised unreported exception
Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/import_utils.py", line 1793, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/usr/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 

TypeError: object of type 'NoneType' has no len()